In [ ]:
import gensim
import numpy as np
import pandas as pd
import pysolr
from gensim.models import Doc2Vec
from transformers import pipeline, set_seed
set_seed(42)
np.random.seed(42)
import sys

In [ ]:
# In case your sys.path does not contain the base repo, cd there.
print(sys.path)
%cd '~/ml-solr-course'

In [ ]:
# Use the pipeline method to invoke the text-generation solution with gpt-2 as model.
generator = None
query = 'Midtown sunny two bedroom'
expanded_queries = None #  Generate 10 possible expansions of 50 characters

In [ ]:
expanded_queries

In [ ]:
model_path = '2-ranking/lab4/airbnb_model'
doc2vec_model = Doc2Vec.load(model_path)
print(f'Doc2Vec Model loaded')

In [ ]:
queries = []
# For each possible expansion, get the similarity with the original query and get the best
for expanded_query in expanded_queries:
    pass
expanded_query = ''

In [ ]:
# Create a client instance.
solr = pysolr.Solr('http://localhost:8983/solr/airbnb', always_commit=True, timeout=10)

In [ ]:
non_expanded_results = solr.search(query, **{
                'rows': 100,
            })


In [ ]:
expanded_results = solr.search(expanded_query, **{
                'rows': 100,
            })

In [ ]:
#Tokenize both the query, get the similarities with both the expanded and not expanded results in the pandas dataframe as always
tokenized_query = None

In [ ]:
df_non_expanded_results = pd.DataFrame(non_expanded_results)
# Fill
df_non_expanded_results.sort_values(by="Similarity", ascending=False, inplace=True)


In [ ]:
print(f'Most similar document before expansion has description: \n\n{df_non_expanded_results["description"].iloc[0]}\n\nWith similarity: {df_non_expanded_results["Similarity"].iloc[0]}')

In [ ]:
df_expanded_results = pd.DataFrame(expanded_results)
# Fill
df_expanded_results.sort_values(by="Similarity", ascending=False, inplace=True)

In [ ]:
print(f'Most similar document after expansion has description: \n\n{df_expanded_results["description"].iloc[0]}\n\nWith similarity: {df_expanded_results["Similarity"].iloc[0]}')

In [ ]:
print(f'Number of documents that surpass 0.5 similarity threshold: {len(df_expanded_results[df_expanded_results["Similarity"] >= 0.5])}')


So it was useless? A key part of our models is that they are trained with our data. I leave you as exercise to see how much it improves if you finetune gpt-2 with a simple dense layer and train that over our airbnb dataset.

Other problem to fix is that our airbnb similarity model is based on our airbnb data, but gpt-2 is generic. We should evaluate word similarity in the same terms, with the glove vectors alone instead. I leave it to you as exercise to see how much that changes as well.


